<center>Universidade Federal de Viçosa</center>
<center>Inteligência Artificial e Computacional</center>
<center>ELT579 - Tópicos Especiais em Inteligência Artificial</center>
<center>TURMA 2022/2</center>

**<center>Semana 4: Detecção de malária em imagens de células </center>**
<center>Aluno: Michel Batistin Fiorio</center>

**Objetivo:**
Durante as aulas da quarta semana do curso foi implementada uma CNN para classificação de imagens de células humanas em dois grupos, o de células com malária e de células sem malária. 

Utilizando um banco de dados de 2000 imagens de células (1000 células doentes e 1000 células saudáveis) o algoritmo desenvolvido em aula obteve uma precisão de 96% na inferência das imagens de teste. 

O objetivo dessa atividade é implementar melhorias no algoritmo para que o resultado do modelo supere o resultado obtido em aula.

**Alterações relevantes realizadas no modelo final:**
- Utilização do banco de dados completo disponível. 22558 imagens, sendo metade das imagens de células positivas, metade de células negativas. Destas, foram realizadas os seguintes agrupamentos:
    - 18047 imagens utilizadas para treinamento
    - 4511 imagens de validação
    - 5000 imagens de teste
- Foram adicionados no modelo camadas de aumento de dados. As camadas utilizadas foram RandomFlip, RandomRotation, RandomZoom.
- Foram adicionadas 2 camadas convolucionais de 32 filtros. 
- Foi alterada a função de ativação da camada de saída para sigmoid e reduzida de 2 para 1 neurônio.
- Foi alterada a função perda de treinamento para binary_crossentropy.
- Foi adicionado uma função de parada prematura no treinamento da CNN. Configurado para monitorar o 'val_loss' e parar o treinamento após 10 épocas sem redução do índice. 

**Resultados alcançados:**
Após testarmos inumeras configurações de pré-processamento dos dados e na estrutura da CNN, o melhor resultado alcançado na ativididade foi uma precisão de 96,24% nos dados de teste.

In [64]:
# Importando bibliotecas

import numpy as np
import cv2 
import os
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout 
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory

In [26]:
# Criando caminhos para acesso a pasta das imagens

# Caminho das imagens de treinamento e validação
pasta = 'C:/Users/mbfio/imagens_celulas_completo/'

# Caminho das imagens de teste (foram separadas 5000 imagens para essa função)
teste = 'C:/Users/mbfio/imagens_celulas_teste/'


In [ ]:
# Definindo o tamanho das imagens 

SIZE = 64

TAM = (SIZE, SIZE, 3)

In [47]:
# Criando o dataset de treinamento
# Foi utilizado a função 'image_dataset_from_directory' somente para fins de aprendizado

train_ds = image_dataset_from_directory(pasta, validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(SIZE, SIZE),
  batch_size=64)

Found 22558 files belonging to 2 classes.
Using 18047 files for training.


In [48]:
# Criando o dataset de validação, utilizando 20% das fotos da pasta

valid_ds = image_dataset_from_directory(pasta, validation_split=0.2, 
  subset="validation",
  seed=123,
  image_size=(SIZE, SIZE),
  batch_size=64)

Found 22558 files belonging to 2 classes.
Using 4511 files for validation.


In [49]:
# Mostrando o nome das classes do banco de dados

class_names = train_ds.class_names
print(class_names)

['negativos', 'positivos']


In [50]:
# Mostrando o tamanho dos pacotes de imagens e formato das imagens

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 64, 64, 3)
(64,)


In [60]:
# Criando o dataset de teste

test_ds = image_dataset_from_directory(teste, seed=123,
  image_size=(SIZE, SIZE),
  batch_size=50)

Found 5000 files belonging to 2 classes.


In [61]:
# Mostrando o nome das classes do banco de dados de teste

test_ds.class_names

['negativos', 'positivos']

In [62]:
# Mostrando o tamanho dos pacotes de imagens e formato das imagens

for image_batch, labels_batch in test_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(50, 64, 64, 3)
(50,)


In [122]:
# Criando a estrutura da rede neural convolucional

modelo = Sequential()

# Normalização dos dados das imagens
modelo.add(Rescaling(1./255))

# Aumento de dados
modelo.add(RandomFlip("horizontal_and_vertical", seed=0))
modelo.add(RandomRotation(0.2, seed=0))
modelo.add(RandomZoom(0.2, 0.2, fill_mode='nearest', seed=0))

# Camadas convolucionais
modelo.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same', input_shape = TAM))
modelo.add(MaxPooling2D(pool_size = (2,2)))
modelo.add(Dropout(rate = 0.2))
modelo.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
modelo.add(MaxPooling2D(pool_size = (2,2)))
modelo.add(Dropout(rate = 0.2))
modelo.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
modelo.add(MaxPooling2D(pool_size = (2,2)))
modelo.add(Dropout(rate = 0.2))
modelo.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
modelo.add(MaxPooling2D(pool_size = (2,2)))
modelo.add(Dropout(rate = 0.2))
modelo.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
modelo.add(MaxPooling2D(pool_size = (2,2)))
modelo.add(Dropout(rate = 0.2))

modelo.add(Flatten())

# Camadas totalmente conectadas
modelo.add(Dense(512, activation = 'relu'))
modelo.add(BatchNormalization(axis=-1))
modelo.add(Dropout(rate = 0.2))
modelo.add(Dense(256, activation = 'relu'))
modelo.add(BatchNormalization(axis=-1))
modelo.add(Dropout(rate = 0.2))
modelo.add(Dense(1, activation= 'sigmoid'))

# Compilando o modelo
modelo.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [123]:
# Treinamento do modelo

parada = EarlyStopping(monitor = 'val_loss', patience=10, restore_best_weights=True, verbose=True)

history = modelo.fit(train_ds,validation_data=valid_ds, verbose = 1, epochs = 80, callbacks=parada)

Epoch 1/80
282/282 [==============================] - 27s 86ms/step - loss: 0.7090 - accuracy: 0.6200 - val_loss: 1.3141 - val_accuracy: 0.5108
Epoch 2/80
282/282 [==============================] - 24s 84ms/step - loss: 0.6229 - accuracy: 0.6653 - val_loss: 1.1243 - val_accuracy: 0.5108
Epoch 3/80
282/282 [==============================] - 24s 84ms/step - loss: 0.5989 - accuracy: 0.6895 - val_loss: 0.9124 - val_accuracy: 0.5108
Epoch 4/80
282/282 [==============================] - 24s 84ms/step - loss: 0.2290 - accuracy: 0.9208 - val_loss: 0.1705 - val_accuracy: 0.9472
Epoch 5/80
282/282 [==============================] - 24s 85ms/step - loss: 0.1832 - accuracy: 0.9419 - val_loss: 0.1733 - val_accuracy: 0.9470
Epoch 6/80
282/282 [==============================] - 24s 85ms/step - loss: 0.1720 - accuracy: 0.9457 - val_loss: 0.1587 - val_accuracy: 0.9517
Epoch 7/80
282/282 [==============================] - 24s 85ms/step - loss: 0.1663 - accuracy: 0.9489 - val_loss: 0.1627 - val_accuracy:

In [124]:
# Acurácia do modelo treinado nos dados de teste

_,score = modelo.evaluate(test_ds, batch_size=50, verbose=0)

print('Acurária do modelo:', score)

Acurária do modelo: 0.9467999935150146
